<a href="https://colab.research.google.com/github/ViMasHub/Test_Kontur/blob/main/Test_Kasperskiy_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Загрузка библиотек

In [ ]:
import pandas as pd
import numpy as np
import re

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

#Загрузка данных


In [ ]:
df_test = pd.read_excel('/content/Test_check.xlsx')
df_test.head()

,URL,lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп..."


#Предобработка данных


In [ ]:
# Установка библиотеки pymorphy3

!pip install pymorphy3


In [ ]:
# Загрузка библиотек

import nltk
import pymorphy3
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Загружаем набор данных для токенизатора
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = stopwords.words('russian')
punctuation_marks = ['"','!', ',', '(', ')', ':', '-', '—', '?', '.', '..', '...']
morph = pymorphy3.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
len(stop_words)

151

In [ ]:
# Функция предобработки тескта
def preprocess (text, stop_words, punctuation_marks, morph):

  no_num_text = re.sub(r'\d', '', text)

  tokens = word_tokenize(no_num_text.lower())

  preprocessed_text = []

  for token in tokens:

    if token not in punctuation_marks:

      lemma = morph.parse(token)[0].normal_form

      if lemma not in stop_words:

        preprocessed_text.append(lemma)

  return preprocessed_text

In [ ]:
df_test['pre_lib_text'] = pd_test.apply(lambda row: preprocess(row['lib_text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
df_test.head()

,URL,lib_text,pre_lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци..."
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","[москва, фев, риа, новость, ключевой, процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро..."
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"[самозапрет, кредит, это, новый, возможность, ..."
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ..."


#Парсинг эталонов

In [ ]:
#postList = [] # список для извлекаемых данных
#fullData = requests.get('https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/') # запрос
#soup = BeautifulSoup(fullData.text, 'html5lib') # парсинг страницы сайта
#postList = soup.find_all('p')
#postList



In [ ]:
# получаем исходный код страницы
inner_html_code = str(urlopen('https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/').read(), 'utf-8')

# отправляем исходный код страницы на обработку в библиотеку
inner_soup = BeautifulSoup(inner_html_code, "html.parser")

inner_soup = inner_soup.find('div', {'class': 'plain-text'})

# Функция чистки текста
def delete_div(code,tag,arg):
     # находим все указанные теги с параметрами
     for div in code.find_all(tag, arg):
        # и удаляем их из кода
        div.decompose()


delete_div(inner_soup, 'div', '')
delete_div(inner_soup, 'h2', '')
delete_div(inner_soup, 'p', {'class': 'title-red'})
delete_div(inner_soup, 'a', '')

In [ ]:
def extract_reference_text(url):
        response = requests.get(url, timeout=30)
        soup = BeautifulSoup(response.content, 'html.parser')

        for tag in soup.find_all(['script', 'style', 'noscript', 'iframe', 'header', 'footer', 'nav', 'aside']):
            tag.decompose()

        article_content = ' '
        domain = url.split('/')[2]

        if 'ria.ru' in domain:
            selectors = ['.article__body', '.article__text', '.layout-article__main']
        elif 'expert.ru' in domain:
            selectors = ['.article__text', '.article__content']
        elif 'lenta.ru' in domain:
            selectors = ['.topic-body__content', '.news-text']
        else:
            selectors = ['article', '.article', '.content', '.news-content', '.post-content', '.entry-content', 'main']

        # Пытаемся найти контент по селекторам
        content_found = False
        for selector in selectors:
            content = soup.select(selector)
            if content:
                for element in content:
                    article_content += element.get_text() + ' '
                content_found = True
                break

        if not content_found:
            paragraphs = soup.find_all('p')
            for p in paragraphs:
                article_content += p.get_text() + ' '

        article_content = re.sub(r'\s+', ' ', article_content).strip()

        return article_content

In [ ]:
df_test['ref_text'] = None

for i, url in enumerate(df_test['URL']):
        reference_text = extract_reference_text(url)
        df_test.at[i, 'ref_text'] = reference_text



In [ ]:
df_test.head()

,URL,lib_text,pre_lib_text,reference_text,ref_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци...",Серьезнее других от введения западных санкций ...,Серьезнее других от введения западных санкций ...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","[москва, фев, риа, новость, ключевой, процентн...",ОглавлениеКлючевая ставкаИзменение ключевой ст...,ОглавлениеКлючевая ставкаИзменение ключевой ст...
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро...",Со следующего года начнется реализация 12 мега...,Со следующего года начнется реализация 12 мега...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"[самозапрет, кредит, это, новый, возможность, ...",ОглавлениеЧто такое самозапрет на кредитыДля ч...,ОглавлениеЧто такое самозапрет на кредитыДля ч...
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ...","1Что такое гринвошинг?Заходя в магазин, вы час...","1Что такое гринвошинг?Заходя в магазин, вы час..."


In [ ]:
df_test['pre_ref_text'] = df_test.apply(lambda row: preprocess(row['ref_text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
df_test.head()

,URL,lib_text,pre_lib_text,reference_text,ref_text,pre_ref_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци...",Серьезнее других от введения западных санкций ...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци..."
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","[москва, фев, риа, новость, ключевой, процентн...",ОглавлениеКлючевая ставкаИзменение ключевой ст...,ОглавлениеКлючевая ставкаИзменение ключевой ст...,"[оглавлениеключевой, ставкаизменение, ключевой..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро...",Со следующего года начнется реализация 12 мега...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро..."
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"[самозапрет, кредит, это, новый, возможность, ...",ОглавлениеЧто такое самозапрет на кредитыДля ч...,ОглавлениеЧто такое самозапрет на кредитыДля ч...,"[оглавлениечтый, такой, самозапрет, кредитыдли..."
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ...","1Что такое гринвошинг?Заходя в магазин, вы час...","1Что такое гринвошинг?Заходя в магазин, вы час...","[такой, гринвошинг, заходить, магазин, часто, ..."
